In [1]:
import pandas as pd
from configparser import ConfigParser
import psycopg2
import pandas.io.sql as sqlio
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sns
from matplotlib import pyplot
%matplotlib inline
sns.set()

In [2]:
password = "postgres"

In [3]:

param_dic={
    "host": "energy-analysis.cfcrgd6zjkoj.us-east-2.rds.amazonaws.com",
    "database": "Energy_Analysis",
    "user" : "postgres",
    "password" : password
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [ ]:
query="select * from ml_input"

ml_input_df=sqlio.read_sql_query(query,conn)

ml_input_df.head()

In [ ]:
ml_input_df["ev_flag"].value_counts()

In [ ]:
query="select * from tbl_main"

main_df=sqlio.read_sql_query(query,conn)

main_df.head()

In [ ]:
query="select * from tbl_person"

person_df=sqlio.read_sql_query(query,conn)

person_df.head()

In [ ]:
query="select * from tbl_PEV"

pev_df=sqlio.read_sql_query(query,conn)

pev_df.head()

In [ ]:
#Get the individual respondents
respondent_df = person_df.loc[person_df["relationship"]==8]
respondent_df

In [ ]:
#Merge respondents with Main table
respondent_merged_df = main_df.merge(respondent_df, how='left', on='sampno') 
respondent_merged_df

In [ ]:
#Get list of all PEV owners
PEV_owners=pev_df.sampno 
PEV_owners.tolist()
len(PEV_owners)


In [ ]:
#Create PEV owner column, populate from list
respondent_merged_df["PEV owner"] = "No"
for owner in PEV_owners:
    respondent_merged_df.loc[respondent_merged_df.sampno == owner,'PEV owner'] = "Yes"

In [ ]:
#Create new 'Genders' column with 3 labels.
respondent_merged_df["Genders"] =respondent_merged_df[""] = "Other"
respondent_merged_df.loc[respondent_merged_df.gender == 1,'Genders'] = "Male"
respondent_merged_df.loc[respondent_merged_df.gender == 2,'Genders'] = "Female"

In [ ]:
#Seperate df for PEV's and non PEVS (In case we want'em)
owner_merged_df =respondent_merged_df.loc[respondent_merged_df['PEV owner'] == "Yes"]
owner_merged_df

In [ ]:
nonowner_merged_df =respondent_merged_df.loc[respondent_merged_df['PEV owner'] == "No"]
nonowner_merged_df

In [ ]:
#Create Donut of owners v Non-owners
fig, ax = plt.subplots(1,figsize=(10,10)) #ax1,ax2 refer to your two pies
labels = ["No EV vehicle","EV owner"]
values =(respondent_merged_df['PEV owner'].value_counts())
#colors
colors = ["#de8f05", "#0173b2"]
#explsion
explode = (0.0,0.0,0.00)
 
ax.pie(values, colors = colors,wedgeprops={'alpha':0.6}, autopct='%1.1f%%', startangle=90, pctdistance=0.85)
ax.set_title('Survey Respondents EV Vehicle Ownership.', fontsize=18)
#draw circle
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
# Equal aspect ratio ensures that pie is drawn as a circle
#ax.axis('equal')  
ax.legend(labels,bbox_to_anchor=(1, 1),borderpad=2,labelspacing=1.5,prop={"size":15})
plt.tight_layout()
plt.show()

# Compare to populations (PEV/Non-PEV)


In [ ]:
#EV owners by age donut
# Pie chart
fig, ax = plt.subplots(1,figsize=(10,10)) #ax1,ax2 refer to your two pies
labels = ["35 to 64 ","65 or older","18 to 34"]
values =(owner_merged_df['age_grp'].value_counts())
#colors
colors = ['#de8f05','#0173b2','#cc78bc']
#explsion
explode = (0.0,0.0,0.00)
 
ax.pie(values, colors = colors,wedgeprops={'alpha':0.6},autopct='%1.1f%%', startangle=90, pctdistance=0.85)
ax.set_title('Surveyed EV Owners By Age Group', fontsize=18)
#draw circle
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
# Equal aspect ratio ensures that pie is drawn as a circle
#ax1.axis('equal')  
ax.legend(labels,bbox_to_anchor=(1, 1),borderpad=2,labelspacing=1.5,prop={"size":15})
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,figsize=(10,10)) #ax1,ax2 refer to your two pies
labels = ["35 to 64 ","65 or older","18 to 34"]
values =(nonowner_merged_df['age_grp'].value_counts())
#colors
colors = ['#de8f05','#0173b2','#cc78bc']
#explsion
explode = (0.0,0.0,0.00)
 
ax.pie(values, colors = colors,wedgeprops={'alpha':0.6}, autopct='%1.1f%%', startangle=90, pctdistance=0.85)
ax.set_title('Surveyed non-EV Owners By Age Group', fontsize=18)
#draw circle
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
# Equal aspect ratio ensures that pie is drawn as a circle
#ax1.axis('equal')  
ax.legend(labels,bbox_to_anchor=(1, 1),borderpad=2,labelspacing=1.5,prop={"size":15})
plt.tight_layout()
plt.show()

In [ ]:
# Draw a nested barplot income /owner

colors = ["#de8f05", "#0173b2"]
# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))
g = sns.countplot(
    data=respondent_merged_df, 
    x="income", hue= "PEV owner", palette = customPalette,
      alpha=0.6,
)
g.set_xticklabels(["less than $10",
"$10 - $25",
"$25 - $35",
"$35 - $50",
"$50 - $75",
"$75 - $100",
"$100 - $150",
"$150 - $200",
"$200 - $250",
"$250 or more",
"Prefer not to answer"],rotation=90)

#g.despine(left=True)
g.set(xlabel="Annual Income ($k)", ylabel = "")
g.set_yscale("log")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.).set_title("EV Owner")
#g.set_axis_labels("Annual Income($k)", " ")
#g.legend.set_title("")

In [ ]:
#Drop "prefer not to say"
df_income_declared = respondent_merged_df[respondent_merged_df['income'] != 11]

In [ ]:
# Income / PEV owner box plot?
colors = ["#de8f05", "#0173b2"]
# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))

#Income /PEV non-PEV
g= sns.catplot(x="PEV owner", y="income", kind="box",palette=customPalette,boxprops=dict(alpha=.8), data=df_income_declared)
g.set_yticklabels(["",
"$10-$25 ",

"$35 - $50",

"$75 - $100",

"$150 - $200",

"250+",""
])  
g.set(ylabel = "annual income ($k)")

In [ ]:
#Income/Age/ownership violin

colors = ["#de8f05", "#0173b2"]
# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))

g=sns.catplot(x="age_grp", y="income", hue="PEV owner",
            kind="violin",inner="stick",split=True, palette=customPalette,violinprops=dict(alpha=.2),  data=df_income_declared)
plt.setp(ax.collections, alpha=.6)
g.set_xticklabels(['18-34','35-64','65+'])
g.set(xlabel="Age Group", ylabel = "annual income($k)")
g.set_yticklabels(["","",
"$10 - $25",


"$35 - $50",

"$75 - $100",

"$150 - $200",

"250+","",""])


In [ ]:
#Education /ownership.
colors = ["#de8f05","#0173b2"]
customPalette = sns.set_palette(sns.color_palette(colors))
# Set your custom color palette

g=sns.countplot(
    data=respondent_merged_df, 
    x="education",hue="PEV owner",
     alpha=0.6
)
g.set_xticklabels(["Less than high school",
                   "High school graduate","Tech school",
"Some college","Associate degree)",
"College graduate",
"Post-graduate work",
"Post-graduate degree"],rotation=90)
g.set_yscale("log")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.).set_title("EV Owner")
#g.set_title("Education level")
g.set(xlabel="", ylabel = "Respondents(log)")

In [ ]:
#Age Bar chart
colors = ["#de8f05", "#0173b2"]
customPalette = sns.set_palette(sns.color_palette(colors))
g=sns.countplot(
    data=respondent_merged_df, 
    x="age_grp", hue= "PEV owner",
    palette=customPalette, alpha=.6,
)
g.set_xticklabels(["18 to 34", "35 to 64", "65 or older"])
g.set_yscale("log")
g.set(xlabel="Age Group", ylabel = "")

In [ ]:
colors = ["#de8f05", "#0173b2"]
# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))
g=sns.countplot(
    data=respondent_merged_df, 
    x="Genders", hue= "PEV owner",
    palette=customPalette, alpha=.6,
)
#g.set_xticklabels(["18 to 34", "35 to 64", "65 or older"])
g.set(xlabel="", ylabel = "")
g.set_yscale("log")

In [ ]:
colors = ["#de8f05", "#0173b2"]




# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))
g=sns.catplot(x="education", y="income", hue="PEV owner",
            kind="box",palette=customPalette,boxprops=dict(alpha=.8), data=df_income_declared)
g.set_xticklabels(["Less than high school",
                   "High school graduate","Tech school",
"Some college","Associate degree)",
"College graduate",
"Post-graduate work",
"Post-graduate degree"],rotation=90)
g.set_yticklabels(["",
"$10 - $25",
"$35 - $50",
"$75 - $100",
"$150 - $200",
"$250+",""
])
g.set(ylabel="Annual Income ($k)", xlabel = "Education")

In [ ]:
#Got Solar?
g=sns.countplot(
    data=respondent_merged_df, 
    x="solar", hue= "PEV owner",
    palette="dark", alpha=.6,
)
g.set_xticklabels(["yes", "no"])
g.set(xlabel="Solar", ylabel = "")
#g.set_yscale("log")

In [ ]:
# 'autonomous_hhveh' autonomous adoption.
#1.We would be one of the first to buy a self-driving vehicle (either as a replacement or additional household vehicle)
#2.We would eventually buy a self-driving vehicle, but only after they are in common use
#3.We would wait as long as possible and try to avoid ever buying a self-driving vehicle
colors = ["#de8f05", "#0173b2"]
# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))
g=sns.countplot(
    data=respondent_merged_df, 
    x='autonomous_hhveh', hue= "PEV owner",
    palette=customPalette, alpha=.6,
)
g.set_xticklabels(["We'd be one of the first !", "We would eventually", "We'd wait as long as possible!"],rotation=90)
g.set(xlabel="", ylabel = "")
#g.set_yscale("log")

In [ ]:
#Print palette hex codes.
pal = sns.color_palette("colorblind")
print(pal.as_hex())

In [ ]:
#Gender age income central tendency (Drop no response income)
colors = ["#de8f05", "#0173b2"]
customPalette = sns.set_palette(sns.color_palette(colors))
data = respondent_merged_df.loc[respondent_merged_df.income != 11]
g = sns.catplot( x="Genders", y="income",
                hue="PEV owner", col="age_grp",
                data=data, kind="bar", alpha= 0.6,
                height=4, aspect=.7);

In [ ]:
#Dual-y axis education/ownership
#colors = ["#de8f05", "#0173b2"]
df_n=respondent_merged_df.loc[respondent_merged_df["PEV owner"]=="No"]
df_y=respondent_merged_df.loc[respondent_merged_df["PEV owner"]=="Yes"]
no_counts = df_n["education"].value_counts().sort_index()
yes_counts =df_y["education"].value_counts().sort_index()

fig = plt.figure() # Create matplotlib figure
ax = fig.add_subplot(111) # Create axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4
no_counts.plot(kind='bar', color="#de8f05",alpha=0.6, ax=ax, width=width,position=1)
yes_counts.plot(kind='bar', color="#0173b2",alpha=0.6, ax=ax2, width=width, position=0)
ax.grid(False) 
ax2.grid(False)
ax.set_ylabel('Non-PEV owners',color="#de8f05")
ax2.set_ylabel('PEV owners',color="#0173b2")

ax.set_xticklabels(["Less than high school",
                   "High school graduate","Tech school",
"Some college","Associate degree)",
"College graduate",
"Post-graduate work",
"Post-graduate degree"],rotation=90)
plt.xlim([-.5,7.5])
plt.show()

In [ ]:
no_counts

In [ ]:
#Dual-y axis gender/ownership
#colors = ["#de8f05", "#0173b2"]
df_n=respondent_merged_df.loc[respondent_merged_df["PEV owner"]=="No"]
df_y=respondent_merged_df.loc[respondent_merged_df["PEV owner"]=="Yes"]
no_counts_inc = df_n["income"].value_counts().sort_index()
yes_counts_inc =df_y["income"].value_counts().sort_index()

d={1:0}# make empty dic. for missing value
fixed_inc_count=pd.Series(d).append(yes_counts_inc)# add dict. to series

fig = plt.figure() # Create matplotlib figure
ax = fig.add_subplot(111) # Create axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4
no_counts_inc.plot(kind='bar', color="#de8f05",alpha=0.6, ax=ax, width=width,position=1)
fixed_inc_count.plot(kind='bar', color="#0173b2",alpha=0.6, ax=ax2, width=width, position=0)
ax.grid(False) 
ax2.grid(False)
ax.set_ylabel('Non-PEV owners',color="#de8f05")
ax2.set_ylabel('PEV owners',color="#0173b2")
ax.set_xlabel("Income($k)")
ax.set_xticklabels(["less than $10",
"$10 - $25",
"$25 - $35",
"$35 - $50",
"$50 - $75",
"$75 - $100",
"$100 - $150",
"$150 - $200",
"$200 - $250",
"$250 or more",
"Prefer not to answer"],rotation=90)
plt.xlim([-.5,10.5])
plt.show()

In [ ]:
#Dual-y axis education/ownership
#colors = ["#de8f05", "#0173b2"]
df_n=respondent_merged_df.loc[respondent_merged_df["PEV owner"]=="No"]
df_y=respondent_merged_df.loc[respondent_merged_df["PEV owner"]=="Yes"]
no_counts = df_n["Genders"].value_counts().sort_index()
yes_counts =df_y["Genders"].value_counts().sort_index()

fig = plt.figure() # Create matplotlib figure
ax = fig.add_subplot(111) # Create axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = .4
no_counts.plot(kind='bar', color="#de8f05",alpha=0.6, ax=ax, width=width,position=1,align='center')
yes_counts.plot(kind='bar', color="#0173b2",alpha=0.6, ax=ax2, width=width, position=0,align='center')
ax.grid() 
ax2.grid()
ax.set_ylabel('Non-PEV owners',color="#de8f05")
ax2.set_ylabel('PEV owners',color="#0173b2")
plt.xlim([-.5,2.5])
ax.set_xticklabels(["Female","Male","Other"],rotation=0)
plt.show()

In [ ]:
no_counts

In [ ]:
df_n=respondent_merged_df.loc[respondent_merged_df["PEV owner"]=="No"]
df_y=respondent_merged_df.loc[respondent_merged_df["PEV owner"]=="Yes"]
no_counts_hhveh = df_n["autonomous_hhveh"].value_counts().sort_index()
yes_counts_hhveh =df_y["autonomous_hhveh"].value_counts().sort_index()

fig = plt.figure() # Create matplotlib figure
ax = fig.add_subplot(111) # Create axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4
no_counts_hhveh.plot(kind='bar', color="#de8f05",alpha=0.6, ax=ax, width=width,position=1)
yes_counts_hhveh.plot(kind='bar', color="#0173b2",alpha=0.6, ax=ax2, width=width, position=0)
ax.grid() 
ax2.grid()
ax.set_ylabel('Non-PEV owners',color="#de8f05")
ax2.set_ylabel('PEV owners',color="#0173b2")
ax.set_xlabel("")
ax.set_xticklabels(["We'd be one of the first !", "We would eventually", "We'd wait as long as possible!"],rotation=90)
plt.xlim([-.5,2.5])
#plt.grid(axis = 'y')

sns.set_style("whitegrid")

plt.show()